In [1]:
from eq1 import config
from main import run

In [2]:
run(config)

14:56:52.775 [INFO] Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
14:56:52.776 [INFO] Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Interpreter Host
14:56:52.777 [INFO] Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
14:56:53.297 [INFO] Constructed symbolic model
14:56:53.380 [INFO] Integrated
14:57:01.175 [INFO] Lambdified
14:57:01.176 [INFO] Constructed JAXified model
14:57:01.984 [INFO] W₀ = [ 0.0006,  0.0005,  0.0003,  0.0006,  0.0004,  0.0007,  0.0005,  0.0004,  0.0001,  0.0006,  0.0005,  0.0001,  0.0002,  0.0008,  0.0001,  0.0009,  0.0006,  0.0008,  0.0003,  0.0005,  0.0004,  0.0009,  0.0000,  0.0006,  0.0007,  0.0006,  0.0010,  0.0006,  0.0004,  0.0002,  0.0007,  0.0007,  0.0004,  0.0008,  0.0006,  0.0005,  0.0008,  0.0002,  0.0000,  0.0000]
14:57:03.734 [INFO] Epoch: 1, Loss: 5.862129211425781,	W = [ 0.

Nothing more to prune!


$\displaystyle - 4.71931025201103 \cdot 10^{-7} k x - 6.32445791672626 \cdot 10^{-7} e^{k} - 8.16085827786288 \cdot 10^{-7} e^{0.000654831412248313 k x} + 0.00100287851819303$